In [ ]:
import os
import time
import random
from itertools import count, combinations

import chess
import pandas as pd

from IPython.display import clear_output

In [ ]:
bot_dir = 'bots'
bot_files = [bot_file for bot_file in os.listdir(bot_dir) if bot_file.endswith('.py')]
bot_files = sorted(bot_files)

bot_files

In [ ]:
# import bots from the files

bot_classes = []

for bot_file in bot_files:
    bot_name = bot_file[:-3]
    bot_module = __import__('bots.' + bot_name, fromlist=[bot_name])
    bot_class = getattr(bot_module, "ChessBot")
    bot_class.name = bot_name
    
    bot_classes.append(bot_class)

print(bot_classes)

In [ ]:
class Judge():
    def __init__(self, player_1, player_2, time_limit=300): # 5 minutes per player
        self.player_1 = player_1            # white
        self.player_2 = player_2            # black
        self.time_limit = time_limit

    def run_game(self, initial_board_fen: str = None, slow_down: bool = False, verbose: bool = False):
        if initial_board_fen is None:
            initial_board_fen = chess.Board().fen()
        board = chess.Board(initial_board_fen)

        player_times = [0, 0]
        notes = []
        winner = None

        for i in count(0, 1):
            if board.is_checkmate():
                winner = "black" if i%2 == 0 else "white"
                notes.append(f"Checkmate! Winner is {winner}")
                break
            elif board.is_stalemate():
                notes.append("Stalemate! It's a draw.")
                break
            elif board.is_insufficient_material():
                notes.append("Insufficient material! It's a draw.")
                break
            elif board.is_seventyfive_moves():
                notes.append("Seventy-five moves without a capture or pawn move! It's a draw.")
                break
            elif board.is_fivefold_repetition():
                notes.append("Fivefold repetition! It's a draw.")
                break
            elif board.is_game_over():
                notes.append("Game over! It's a draw.")
                break

            board_fen = board.fen()

            start = time.time()
            if i % 2 == 0:
                move = self.player_1(board_fen)
            else:
                move = self.player_2(board_fen)
            end = time.time()

            player_times[i%2] += end - start

            if player_times[i%2] > self.time_limit:
                winner = "black" if i%2 == 0 else "white"
                notes.append(f"\nTime limit exceeded! Winner is {winner}")
                break

            if not board.is_legal(move):
                winner = "black" if i%2 == 0 else "white"
                hallucinator = "white" if i%2 == 0 else "black"
                notes.append(f"Illegal board move. The {hallucinator} is hallucinating.")
                break

            board.push(move)

            if verbose: 
                clear_output(wait=True)
                display(board)

            # slow down the bots so that we can see them
            if slow_down: 
                time.sleep(.001)

        results = {}
        results['winner'] = winner
        results['times'] = player_times
        results["notes"] = notes

        return results

In [ ]:
with open('opennings.txt', 'r') as file:
    opennings = file.read().split('\n')

In [ ]:
# play all bots against each other

N_GAMES = 3 # per pair color (so N_GAMES * 2 games per pair of bots)

game_results = pd.DataFrame(columns=['white_bot', 'black_bot', 'winner', 'white_time', 'black_time', 'notes'])

for bot1, bot2 in combinations(bot_classes, 2):
    for white, black in [(bot1, bot2), (bot2, bot1)]:
        for i in range(N_GAMES):
            judge = Judge(white(), black())
            random_opening = random.choice(opennings)
            results = judge.run_game(random_opening, verbose=True)

            game_results.loc[len(game_results)] = [
                white.name, black.name, results['winner'], 
                results['times'][0], results['times'][1], 
                "; ".join(results['notes'])
            ]

display(game_results)

In [ ]:
leaderboard = pd.DataFrame(columns=['bot', 'wins', 'draws', 'losses', 'score'])

for bot in bot_classes:
    bot_name = bot.name

    bot_wins = sum((game_results["white_bot"] == bot_name) & (game_results["winner"] == "white")) + \
               sum((game_results["black_bot"] == bot_name) & (game_results["winner"] == "black"))
    
    bot_draws = sum((game_results["white_bot"] == bot_name) & (game_results["winner"].isna())) + \
                sum((game_results["black_bot"] == bot_name) & (game_results["winner"].isna()))
    
    bot_losses = sum((game_results["white_bot"] == bot_name) & (game_results["winner"] == "black")) + \
                    sum((game_results["black_bot"] == bot_name) & (game_results["winner"] == "white"))
    
    bot_score = bot_wins + 0.5 * bot_draws + 0 * bot_losses

    leaderboard.loc[len(leaderboard)] = [bot_name, bot_wins, bot_draws, bot_losses, bot_score]

leaderboard = leaderboard.sort_values(by='score', ascending=False)
leaderboard = leaderboard.reset_index(drop=True)

leaderboard

In [ ]:
today = time.strftime("%Y-%m-%d")
today_results = f"results/{today}_results.csv"
today_leaderboard = f"results/{today}_leaderboard.csv"

game_results.to_csv(today_results, index=False)
leaderboard.to_csv(today_leaderboard, index=False)